# Simulating background selection on a nonrecombining Y chromosome


This notebook uses the packages [fwdpp](https://github.com/molpopgen/fwdpp), [fwdpy](https://github.com/molpopgen/fwdpy), [libsequence](https://github.com/molpopgen/libsequence), and [pylibseq](https://github.com/molpopgen/pylibseq), developed by Kevin Thornton (UC Irvine) for conducting forward-time population genetic simulations and analyzing their output. If you use these packages, please cite:
   * Thornton, K. R. (2014). A C++ template library for efficient forward-time population genetic simulation of large populations. Genetics 98:157-166


   * Thornton, K.R. (2003). Libsequence, a C++ class library for evolutionary genetic analysis. Bioinformatics 19:2325-2327

## Setting up the simulation

* These simulations are aimed at evaluating whether the observed reduction in nucleotide diversity on the _R. hastatulus_ Y-chromosome (relative to neutral evolution predictions) can be explained by the effects of background selection arising due to the lack of recombination on this chromosome. 


* We first simulate and collect diversity statistics for a non-recombining Y chromosome evolving under background selection using a model for the distribution of fitness effects of new mutations in which selective coefficients are drawn from a gamma distribution with shape parameter 0.1. This is implemented by using in the GammaS function of fwdpy. The simulations are repeated with recombination for X chromosomes and autosomes (where $Ne_X$ = $3/4 Ne_A$) to obtain normalized $X/A$ and $Y/A$ ratios of diversity. These simulated diversity ratios are then compared to empirical data and predictions under neutral evolution, and we estimate the strength of background selection required to explain our observed reduction in Y-linked diversity in the _R. hastatulus_ Y chromosome.

In [1]:
#import required libraries
from __future__ import print_function
import fwdpy as fp
import numpy as np
import pandas
import math

## Establishing regions for mutation and recombination on the Y-chromosome


* Neutral mutations occur on the interval $[0,1)$.
* Strongly-deleterious mutations occur on the intervals $[-1,0)$ and $[1,2)$.
* Recombination is uniform throughout the region. Here we set the recombination rate $r=0$ to simulate the case of a nonrecombining Y-chromosome. For the X chromosome and autosomes (below), we allow free recombination with $r=0.5$. Here, $r$ is the mean Poisson number of crossover events (per diploid, per generation) and represents the total rate across the simulated region.

In [2]:
# Where neutral mutations occur:
nregions = [fp.Region(beg=0,end=1,weight=1)]

Our "selected" mutations will have positions on the continuous interval $[1,2)$. There may be two classes of such mutations (deleterious or beneficial), each with gamma-distrubted selection coefficients. Here we are interested in the effects of selection against deleterious mutations, so we set this class to have a mean $s = -0.1$ (deleterious) with weight 1 (all mutations are deleterious)

In [3]:
# Where selected mutations occur:

#constant model with s=-0.05 (not used)
#sregions = [fp.ConstantS(beg=-1,end=0,weight=1,s=-0.05,h=1),
#            fp.ConstantS(beg=1,end=2,weight=1,s=-0.05,h=1)]

#gamma model
sregions = [fp.GammaS(1,2,1,-0.01,0.1,1)] #these are deleterious with mean s= -0.01
           
#input paramters
#b: the beginning of the region
#e: the end of the region
#w: the “weight” assigned to the region
#mean: mean of the Gamma
#shape: shape of the Gamma
#h: the dominance term

In [4]:
# Recombination:
recregions = [fp.Region(beg=-1,end=2,weight=1)]

## Population size and simulation length

In [5]:
#Population size
#we set Ne for tha Y to be 1/4 Ne that of autosomes
N=250
#We'll evolve for 10N generations.
#nlist is a list of population sizes over time.
#len(nlist) is the length of the simulation
#We use numpy arrays for speed and optimised RAM
#use.  Note the dtype=np.uint32, which means 32-bit
#unsigned integer. Failure to use this type will
#cause a run-time error.
nlist = np.array([N]*10*N,dtype=np.uint32)

In [6]:
#Initalize a random number generator with random seed value
rng = fp.GSLrng(238)

In [7]:
#Simulate 24 replicate populations. This uses C++11 threads behind the scenes:
pops = fp.evolve_regions(rng,       #The random number generator
                         24,         #The number of pops to simulate, chosen to be the same as our empirical popns sampled.
                         N,         #Initial population size for each of the populations
                         nlist[0:], #List of population sizes over time.
                         0.005,     #Neutral mutation rate (per gamete, per generation)
                         0.01,      #Deleterious mutation rate (per gamete, per generation)
                         0,         #Recombination rate (per diploid, per generation)
                         nregions,  #Defined above
                         sregions,  #Defined above
                         recregions)#Defined above

In [8]:
#Now, pops is a Python list with len(pops) = 24
#Each element's type is fwdpy.singlepop
print(len(pops))
print(type(pops[0]))

24
<type 'fwdpy.fwdpy.singlepop'>


## Taking samples from simulated populations


In [9]:
#Use a list comprehension to get a random sample of size
#n = 24 from each replicate
samples = [fp.get_samples(rng,i,24) for i in pops]

#Samples is now a list of tuples of two lists.
#Each list contains tuples of mutation positions and genotypes.
#The first list represents neutral variants.
#The second list represents variants affecting fitness ('selected' variants)

for i in samples[:24]:
    print ("A sample from a population is a ",type(i))
    
print(len(samples))

A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type 'tuple'>
A sample from a population is a  <type '

## Getting additional information about samples

In [10]:
#Again, use list comprehension to get the 'details' of each sample
#Given that each object in samples is a tuple, and that the second
#item in each tuple represents selected mutations, i[1] in the line
#below means that we are getting the mutation information only for
#selected variants
details = [fp.get_sample_details(i[1],j) for i,j in zip(samples,pops)]

In [11]:
#details is now a list of pandas DataFrame objects
#Each DataFrame has the following columns:
#  a: mutation age (in generations)
#  h: dominance of the mutation
#  p: frequency of the mutation in the population
#  s: selection coefficient of the mutation
#  label: A label applied for mutations for each region.  Here, I use 0 for all regions
for i in details:
    print(i)

      a  h  label      p             s
0    16  1      0  0.022 -2.976163e-05
1     9  1      0  0.008 -1.544667e-05
2    11  1      0  0.036 -1.016287e-03
3   119  1      0  0.038 -2.200103e-09
4     9  1      0  0.018 -1.908197e-08
5   257  1      0  0.572 -3.569084e-07
6   764  1      0  0.912 -1.720518e-24
7   237  1      0  0.206 -2.334741e-15
8   372  1      0  0.088 -1.853901e-09
9   708  1      0  0.088 -3.627299e-05
10  210  1      0  0.088 -9.895452e-19
11   47  1      0  0.036 -7.760414e-09
12  226  1      0  0.064 -2.182423e-22
13  142  1      0  0.066 -3.067926e-03
14   35  1      0  0.118 -5.873448e-04
15    9  1      0  0.014 -1.854193e-05
16  674  1      0  0.088 -8.351621e-11
17  152  1      0  0.206 -9.075202e-08
18  681  1      0  0.912 -3.429996e-05
19   72  1      0  0.036 -2.714552e-04
20  482  1      0  0.912 -3.228159e-04
21  616  1      0  0.912 -9.035937e-12
22   41  1      0  0.066 -8.690730e-05
23   63  1      0  0.098 -5.719409e-03
24   53  1      0  0.058 

In [12]:
#The order of the rows in each DataFrame is the
#same as the order as the objects in 'samples':
for i in range(24):
    print("Number of sites in samples[",i,"] = ",
          len(samples[i][1]),". Number of rows in DataFrame ",i,
          " = ",len(details[i].index),sep="")

Number of sites in samples[0] = 36. Number of rows in DataFrame 0 = 36
Number of sites in samples[1] = 21. Number of rows in DataFrame 1 = 21
Number of sites in samples[2] = 23. Number of rows in DataFrame 2 = 23
Number of sites in samples[3] = 35. Number of rows in DataFrame 3 = 35
Number of sites in samples[4] = 36. Number of rows in DataFrame 4 = 36
Number of sites in samples[5] = 22. Number of rows in DataFrame 5 = 22
Number of sites in samples[6] = 27. Number of rows in DataFrame 6 = 27
Number of sites in samples[7] = 39. Number of rows in DataFrame 7 = 39
Number of sites in samples[8] = 29. Number of rows in DataFrame 8 = 29
Number of sites in samples[9] = 31. Number of rows in DataFrame 9 = 31
Number of sites in samples[10] = 32. Number of rows in DataFrame 10 = 32
Number of sites in samples[11] = 30. Number of rows in DataFrame 11 = 30
Number of sites in samples[12] = 25. Number of rows in DataFrame 12 = 25
Number of sites in samples[13] = 29. Number of rows in DataFrame 13 = 2

In [13]:
#Add a column to each DataFrame
#specifying the mutation position,
#count of derived state,
#and a "replicate ID"
for i in range(len(details)):
    ##samples[i][1] again is the selected mutations in the sample taken
    ##from the i-th replicate
    details[i]['pos']=[x[0] for x in samples[i][1]]               #Mutation position
    details[i]['count']=[ x[1].count('1') for x in samples[i][1]] #No. occurrences of derived state in sample
    details[i]['id']=[i]*len(details[i].index)                    #Replicate id

In [14]:
##Merge into 1 big DataFrame:
BigTable = pandas.concat(details)

print("This is a merged table:")
print(BigTable)

This is a merged table:
      a  h  label      p             s       pos  count  id
0    16  1      0  0.022 -2.976163e-05  1.021014      2   0
1     9  1      0  0.008 -1.544667e-05  1.033893      2   0
2    11  1      0  0.036 -1.016287e-03  1.087374      1   0
3   119  1      0  0.038 -2.200103e-09  1.142919      1   0
4     9  1      0  0.018 -1.908197e-08  1.151537      1   0
5   257  1      0  0.572 -3.569084e-07  1.162582     12   0
6   764  1      0  0.912 -1.720518e-24  1.172417     23   0
7   237  1      0  0.206 -2.334741e-15  1.198282      5   0
8   372  1      0  0.088 -1.853901e-09  1.212734      1   0
9   708  1      0  0.088 -3.627299e-05  1.214916      1   0
10  210  1      0  0.088 -9.895452e-19  1.228852      1   0
11   47  1      0  0.036 -7.760414e-09  1.244013      1   0
12  226  1      0  0.064 -2.182423e-22  1.254407      2   0
13  142  1      0  0.066 -3.067926e-03  1.276593      4   0
14   35  1      0  0.118 -5.873448e-04  1.277810      2   0
15    9  1      

## Summary statistics from samples

In [15]:
import libsequence.polytable as polyt
import libsequence.summstats as sstats

#Convert neutral mutations into libsequence "SimData" objects,
#which are intended to handle binary (0/1) data like
#what comes out of these simulations
n = [polyt.simData(i[0]) for i in samples]

#Create "factories" for calculating the summary stats
an = [sstats.polySIM(i) for i in n]

##Collect a bunch of summary stats into a pandas.DataFrame:
NeutralMutStats = pandas.DataFrame([ {'thetapi':i.thetapi(),'npoly':i.numpoly(),'thetaw':i.thetaw()} for i in an ])

NeutralMutStats

,npoly,thetapi,thetaw
0,17,2.782609,4.552403
1,12,3.431159,3.213461
2,12,3.231884,3.213461
3,18,4.369565,4.820191
4,22,7.322464,5.891345
5,17,6.097826,4.552403
6,22,4.416667,5.891345
7,28,9.003623,7.498076
8,17,2.202899,4.552403
9,12,1.648551,3.213461


 Now, let's get the average from 1000 simulated replicates. We already have 24 replicates that we did above, so we'll run another 24 sets of four populations. We will use standard Python to grow our collection of summary statistics.

In [16]:
for i in range(0,100,1):
    pops = fp.evolve_regions(rng,
                         24,
                         N,
                         nlist[0:],
                         0.005,
                         0.01,
                         0,
                         nregions,
                         sregions,
                         recregions)
    samples = [fp.get_samples(rng,i,24) for i in pops]
    simdatasNeut = [polyt.simData(i[0]) for i in samples]
    polySIMn = [sstats.polySIM(i) for i in simdatasNeut]
    ##Append stats into our growing DataFrame:
    NeutralMutStats=pandas.concat([NeutralMutStats,
                                   pandas.DataFrame([ {'thetapi':i.thetapi(),
                                                       'npoly':i.numpoly(),
                                                       'thetaw':i.thetaw()} for i in polySIMn ])])

## Getting the mean diversity

We’ve collected everything into a big pandas DataFrame. We can now get the mean using groupby and mean functions.

In [17]:
#Get means for each column:
NeutralMutStats.mean(0)

npoly      16.401403
thetapi     4.199356
thetaw      4.392106
dtype: float64

The ‘thetapi’ record is our mean π from all of the simulations.

Now print the data to a csv file

In [18]:
NeutralMutStats.to_csv('y_new.csv')

## The average $\pi$ under the model
 
Under the BGS model, the expectation of $\pi$ is $E[\pi]=\pi_0e^{-\frac{U}{2sh+r}},$ $U$ is the mutation rate to strongly-deleterious variants, $\pi_0$ is the value expected in the absence of BGS (_i.e._ $\pi_0 = \theta = 4N_e\mu$), $s$ and $h$ are the selection and dominance coefficients, and $r$ is the recombination rate.
 
Note that the definition of $U$ is _per diploid_, meaning twice the per gamete rate. (See Hudson and Kaplan (1995) PMC1206891 for details).

For our parameters, we have $E[\pi] = 5e^{-\frac{0.02}{0.1}},$ which equals:



In [19]:
print(5*math.exp(-0.02/(0.1)))

4.09365376539
